Link to another tutorial site that I was using to figure this out:
https://towardsdatascience.com/deep-neural-networks-for-regression-problems-81321897ca33

In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn import preprocessing #Convert categorical data to neumeric values

from keras.callbacks import ModelCheckpoint
from keras.models import Sequential
from keras.layers import Dense, Activation, Flatten
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error 
from matplotlib import pyplot as plt
import seaborn as sb
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import warnings 
warnings.filterwarnings('ignore')
warnings.filterwarnings('ignore', category=DeprecationWarning)
from xgboost import XGBRegressor

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/kaggle/input/widsdatathon2023/train_data.csv
/kaggle/input/widsdatathon2023/test_data.csv
/kaggle/input/widsdatathon2023/sample_solution.csv


In [2]:
keep_columns=['lat','month','contest-tmp2m-14d__tmp2m',
              'climateregions__climateregion','lon']
X_columns=[['lat','month',
              'climateregions__climateregion','lon']]

In [3]:
def seperate_monthly(df):
    df[['month','day','year']]=df.startdate.str.split('/',expand=True)
    df=df.drop('day', axis=1)
    df=df.drop('startdate',axis=1)
    return df

In [4]:
def eliminate_columns(df):
    for col in df.columns:
        if col in keep_columns:
            print(col)
        else: 
            df=df.drop(col, axis=1)
    return df

In [5]:
#need to come up with a different way to do this, because it is not working
def seperate_y(df):
    y_df=df['contest-tmp2m-14d__tmp2m']
    X_df=df.drop('contest-tmp2m-14d__tmp2m',axis=1)
    return y_df,X_df

In [6]:
def normanlize_data(df,train_df):
    for column in df.columns:
        df[column] = df[column]  /  train_df[column].abs().max()
    return df

In [7]:
train_df=pd.read_csv('/kaggle/input/widsdatathon2023/train_data.csv')
train_df=train_df.dropna(axis=0)
test_df=pd.read_csv('/kaggle/input/widsdatathon2023/test_data.csv')
test_df=test_df.dropna(axis=0)
train_df

,index,lat,lon,startdate,contest-pevpr-sfc-gauss-14d__pevpr,nmme0-tmp2m-34w__cancm30,nmme0-tmp2m-34w__cancm40,nmme0-tmp2m-34w__ccsm30,nmme0-tmp2m-34w__ccsm40,nmme0-tmp2m-34w__cfsv20,...,wind-vwnd-925-2010-11,wind-vwnd-925-2010-12,wind-vwnd-925-2010-13,wind-vwnd-925-2010-14,wind-vwnd-925-2010-15,wind-vwnd-925-2010-16,wind-vwnd-925-2010-17,wind-vwnd-925-2010-18,wind-vwnd-925-2010-19,wind-vwnd-925-2010-20
0,0,0.0,0.833333,9/1/14,237.00,29.02,31.64,29.57,30.73,29.71,...,-27.68,-37.21,8.32,9.56,-2.03,48.13,28.09,-13.50,11.90,4.58
1,1,0.0,0.833333,9/2/14,228.90,29.02,31.64,29.57,30.73,29.71,...,-21.13,-36.57,8.77,21.17,4.44,48.60,27.41,-23.77,15.44,3.42
2,2,0.0,0.833333,9/3/14,220.69,29.02,31.64,29.57,30.73,29.71,...,-10.72,-34.16,6.99,32.16,5.01,48.53,19.21,-33.16,15.11,4.82
3,3,0.0,0.833333,9/4/14,225.28,29.02,31.64,29.57,30.73,29.71,...,0.33,-31.04,6.17,39.66,-1.41,50.59,8.29,-37.22,18.24,9.74
4,4,0.0,0.833333,9/5/14,237.24,29.02,31.64,29.57,30.73,29.71,...,9.83,-31.80,7.47,38.62,-5.21,54.73,-2.58,-42.30,21.91,10.95
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
375729,375729,1.0,0.866667,8/27/16,312.05,23.13,27.20,20.25,24.43,18.35,...,-15.64,-75.68,-3.09,6.93,-16.69,16.98,-13.85,50.25,-31.33,0.77
375730,375730,1.0,0.866667,8/28/16,305.82,23.13,27.20,20.25,24.43,18.35,...,-7.59,-76.42,-13.55,13.36,-15.96,20.45,-16.36,51.65,-30.73,10.10
375731,375731,1.0,0.866667,8/29/16,311.62,23.13,27.20,20.25,24.43,18.35,...,-6.25,-70.65,-23.93,22.62,-16.71,20.28,-15.48,48.58,-18.74,9.28
375732,375732,1.0,0.866667,8/30/16,304.54,23.13,27.20,20.25,24.43,18.35,...,-7.16,-57.67,-33.55,32.06,-16.07,16.60,-20.61,39.23,-16.26,-0.22


In [8]:
#Shuffeling the data
train_df = train_df.sample(frac = 1)

In [9]:
le = preprocessing.LabelEncoder()

# Converting string labels into numbers.
train_df['climateregions__climateregion']=le.fit_transform(train_df['climateregions__climateregion'])
test_df['climateregions__climateregion']=le.fit_transform(test_df['climateregions__climateregion'])

In [10]:
print('training set prep')
train_df=seperate_monthly(train_df)
#train_df=eliminate_columns(train_df)
print('test set prep')
test_df=seperate_monthly(test_df)
#test_df=eliminate_columns(test_df)

training set prep
test set prep


In [11]:
train_df

,index,lat,lon,contest-pevpr-sfc-gauss-14d__pevpr,nmme0-tmp2m-34w__cancm30,nmme0-tmp2m-34w__cancm40,nmme0-tmp2m-34w__ccsm30,nmme0-tmp2m-34w__ccsm40,nmme0-tmp2m-34w__cfsv20,nmme0-tmp2m-34w__gfdlflora0,...,wind-vwnd-925-2010-13,wind-vwnd-925-2010-14,wind-vwnd-925-2010-15,wind-vwnd-925-2010-16,wind-vwnd-925-2010-17,wind-vwnd-925-2010-18,wind-vwnd-925-2010-19,wind-vwnd-925-2010-20,month,year
352409,352409,0.954545,0.666667,58.63,6.45,8.42,6.25,8.53,6.60,9.24,...,-9.93,59.31,-9.88,26.65,-50.41,-42.88,-12.25,10.61,11,14
349926,349926,0.954545,0.533333,18.52,-4.12,-1.28,-3.38,-3.67,-4.20,-0.02,...,-12.79,-11.34,-80.20,-67.04,41.53,-34.16,44.07,-10.11,1,16
216162,216162,0.681818,0.000000,112.82,5.39,6.30,7.38,7.48,6.22,7.50,...,2.51,9.92,-56.24,-42.98,-18.63,-9.28,63.35,14.26,1,16
275799,275799,0.772727,0.866667,196.24,-0.69,0.98,3.47,1.16,0.04,2.88,...,28.65,28.40,-48.89,-16.81,-13.31,-23.04,7.18,45.33,4,15
222001,222001,0.681818,0.266667,11.27,-4.75,-2.11,-2.71,-2.37,-1.49,-1.88,...,-12.79,-11.34,-80.20,-67.04,41.53,-34.16,44.07,-10.11,1,16
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
307085,307085,0.863636,0.466667,122.54,4.63,5.80,3.24,5.66,6.35,6.82,...,-1.21,56.25,-3.74,13.64,-33.96,-36.68,0.23,12.93,11,14
334969,334969,0.909091,0.800000,27.33,-9.46,-10.94,-8.48,-13.63,-9.81,-10.46,...,-37.76,43.11,-74.02,-45.32,-13.35,5.76,0.02,1.71,2,15
68935,68935,0.318182,0.833333,334.56,9.14,12.06,11.55,9.86,9.56,13.50,...,11.25,57.71,-45.77,7.99,-6.87,10.41,13.87,42.03,4,15
309360,309360,0.863636,0.566667,33.61,-5.65,-6.25,-5.70,-5.95,-5.21,-4.50,...,105.16,-32.96,53.33,29.76,14.80,29.69,37.11,-51.49,1,15


In [12]:
test_df

,index,lat,lon,contest-pevpr-sfc-gauss-14d__pevpr,nmme0-tmp2m-34w__cancm30,nmme0-tmp2m-34w__cancm40,nmme0-tmp2m-34w__ccsm30,nmme0-tmp2m-34w__ccsm40,nmme0-tmp2m-34w__cfsv20,nmme0-tmp2m-34w__gfdlflora0,...,wind-vwnd-925-2010-13,wind-vwnd-925-2010-14,wind-vwnd-925-2010-15,wind-vwnd-925-2010-16,wind-vwnd-925-2010-17,wind-vwnd-925-2010-18,wind-vwnd-925-2010-19,wind-vwnd-925-2010-20,month,year
0,375734,0.0,0.833333,339.88,30.88,30.92,29.17,31.02,29.47,30.93,...,-29.25,40.88,-8.31,14.91,-24.62,31.05,-23.69,6.27,11,22
1,375735,0.0,0.833333,334.63,30.88,30.92,29.17,31.02,29.47,30.93,...,-28.62,45.37,-5.42,16.97,-23.94,28.84,-20.61,14.16,11,22
2,375736,0.0,0.833333,337.83,30.88,30.92,29.17,31.02,29.47,30.93,...,-27.67,49.76,-1.31,21.44,-19.06,26.85,-16.78,13.42,11,22
3,375737,0.0,0.833333,345.81,30.88,30.92,29.17,31.02,29.47,30.93,...,-19.32,52.62,-0.44,21.65,-23.12,23.70,-18.62,10.69,11,22
4,375738,0.0,0.833333,357.39,30.88,30.92,29.17,31.02,29.47,30.93,...,-9.89,51.23,-7.57,19.86,-30.56,20.66,-25.08,19.64,11,22
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31349,407083,1.0,0.866667,62.72,4.60,8.71,6.05,10.08,6.39,8.42,...,32.39,38.82,7.42,11.75,-23.62,-0.24,-5.94,51.23,12,22
31350,407084,1.0,0.866667,73.41,4.60,8.71,6.05,10.08,6.39,8.42,...,26.23,37.64,13.01,17.84,-22.05,-3.03,1.31,51.45,12,22
31351,407085,1.0,0.866667,70.00,4.60,8.71,6.05,10.08,6.39,8.42,...,21.06,36.53,14.15,23.12,-25.60,-5.88,9.32,45.32,12,22
31352,407086,1.0,0.866667,79.81,4.60,8.71,6.05,10.08,6.39,8.42,...,20.42,36.05,6.38,29.00,-27.06,-1.42,16.06,31.88,12,22


In [13]:
train_df=train_df.astype('float')
test_df=test_df.astype('float')

In [14]:
y_train_df=train_df['contest-tmp2m-14d__tmp2m']
X_train_df=train_df.drop('contest-tmp2m-14d__tmp2m', axis=1)
type(y_train_df)
y_train_df=y_train_df.to_frame()
type(y_train_df)

pandas.core.frame.DataFrame

In [15]:
X_train_df=normanlize_data(X_train_df,train_df)
test_df=normanlize_data(test_df,train_df)

In [16]:
X_train_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 352604 entries, 352409 to 34976
Columns: 246 entries, index to year
dtypes: float64(246)
memory usage: 664.5 MB


# Model

In [17]:
NN_model = Sequential()

#I might need to change the input_dim to give the actual dimentions
#Instead of being just 1
# The Input Layer :
NN_model.add(Dense(256, kernel_initializer='normal',input_dim = 246, activation='relu'))

# The Hidden Layers :
NN_model.add(Dense(512, kernel_initializer='normal',activation='relu'))
NN_model.add(Dense(512, kernel_initializer='normal',activation='relu'))
NN_model.add(Dense(512, kernel_initializer='normal',activation='relu'))
NN_model.add(Dense(512, kernel_initializer='normal',activation='relu'))

# The Output Layer :
NN_model.add(Dense(1, kernel_initializer='normal',activation='linear'))

# Compile the network :
NN_model.compile(loss='mean_absolute_error', optimizer='adam', metrics=['mean_absolute_error'])
NN_model.summary()


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 256)               63232     
                                                                 
 dense_1 (Dense)             (None, 512)               131584    
                                                                 
 dense_2 (Dense)             (None, 512)               262656    
                                                                 
 dense_3 (Dense)             (None, 512)               262656    
                                                                 
 dense_4 (Dense)             (None, 512)               262656    
                                                                 
 dense_5 (Dense)             (None, 1)                 513       
                                                                 
Total params: 983,297
Trainable params: 983,297
Non-trai

2023-02-22 03:48:04.437743: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda/lib64:/usr/local/cuda/lib:/usr/local/lib/x86_64-linux-gnu:/usr/local/nvidia/lib:/usr/local/nvidia/lib64::/opt/conda/lib
2023-02-22 03:48:04.437804: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)


In [18]:
checkpoint_name = 'Weights--.hdf5' 
checkpoint = ModelCheckpoint(checkpoint_name, monitor='val_loss', verbose = 1, save_best_only = True, mode ='auto')
callbacks_list = [checkpoint]

In [19]:
NN_model.fit(X_train_df, y_train_df, epochs=50, batch_size=32, validation_split = 0.2, callbacks=callbacks_list)

Epoch 1/50
8816/8816 [==============================] - ETA: 0s - loss: 1.0807 - mean_absolute_error: 1.0807
Epoch 1: val_loss improved from inf to 0.96939, saving model to Weights--.hdf5
8816/8816 [==============================] - 84s 10ms/step - loss: 1.0807 - mean_absolute_error: 1.0807 - val_loss: 0.9694 - val_mean_absolute_error: 0.9694
Epoch 2/50
8815/8816 [============================>.] - ETA: 0s - loss: 0.8119 - mean_absolute_error: 0.8119
Epoch 2: val_loss improved from 0.96939 to 0.76777, saving model to Weights--.hdf5
8816/8816 [==============================] - 84s 10ms/step - loss: 0.8119 - mean_absolute_error: 0.8119 - val_loss: 0.7678 - val_mean_absolute_error: 0.7678
Epoch 3/50
8814/8816 [============================>.] - ETA: 0s - loss: 0.7377 - mean_absolute_error: 0.7377
Epoch 3: val_loss improved from 0.76777 to 0.72667, saving model to Weights--.hdf5
8816/8816 [==============================] - 83s 9ms/step - loss: 0.7377 - mean_absolute_error: 0.7377 - val_loss:

In [20]:
# Load wights file of the best model :
wights_file = f'Weights--.hdf5'  # choose the best checkpoint 
NN_model.load_weights(wights_file) # load it
NN_model.compile(loss='mean_absolute_error', optimizer='adam', metrics=['mean_absolute_error'])

In [21]:
predictions = NN_model.predict(test_df)

980/980 [==============================] - 4s 4ms/step


# Saving Dataset in correct format

In [22]:
predictions

array([[26.000134 ],
       [26.070469 ],
       [25.999481 ],
       ...,
       [ 1.5237482],
       [ 1.6766136],
       [ 1.4198928]], dtype=float32)

In [23]:
final=pd.DataFrame(predictions,columns=['contest-tmp2m-14d__tmp2m'])

In [24]:
i=375734
final['index']=0
while i<407088:
    final['index'][i-375734]=i
    i+=1
final

,contest-tmp2m-14d__tmp2m,index
0,26.000134,375734
1,26.070469,375735
2,25.999481,375736
3,26.007757,375737
4,25.967098,375738
...,...,...
31349,2.125636,407083
31350,1.930013,407084
31351,1.523748,407085
31352,1.676614,407086


In [25]:
final.to_csv('/kaggle/working/wetherprediction.csv',index=False)